# Part 12: Analysis of FACS data

This document contains the code used for downstream analysis of Flow Cytometry results, i.e. frequency tables. Upstream analysis was performed in FLowJo software. For details of the flow cytometry analysis, please see the methods section of the manuscript. 

In [ ]:
source("diabetes_analysis_v07.R")


## Proportions of Naive and Effector cells

First, we will see what are the proportions of Naive and Effector cells. We will load the frequencies of populations stored in the metadata file. 

In [ ]:
md <- read_excel("../data/metadata_v07.xlsx")
df <- as.data.frame(md)
df

In [ ]:
df  <- df  %>%  mutate(eff_to_naive_ratio_cd4 = df$pct_eff_of_CD4/df$pct_naive_of_CD4,
                       eff_to_naive_ratio_cd8 = df$pct_eff_of_CD8/df$pct_naive_of_CD8)

Let's plot the ratios ofeffector to naive populations in Cd4 and Cd8 cells. 

In [ ]:
for(i in c(77,78)){
    
# Plot difference between groups (Statistics = Dia T0 vs Ctrl)
  df_filt <- df %>% mutate(Condition = if_else(substr(patient,1,1)=="1","Dia","Ctrl")) %>% 
    mutate(Condition = paste(Condition, time_taken))
     df_filt <- df_filt[,c(1:10,79,i)]
  colnames(df_filt) <- c(colnames(df_filt[1:10]), "Condition","value")
  df_filt$value <- as.numeric(df_filt$value)

    plot  <- df_filt  %>% 
 ggplot(aes(x = Condition, y = value)) + 
 geom_violin(alpha = 0.3, aes(fill = Condition), scale = "width") + 
stat_summary(fun = "median",
               geom = "crossbar", 
               width = 0.75,
               color = "grey30") +
geom_beeswarm(size = 3, aes(fill = Condition), cex = 3, 
                shape = 21, color = "black", corral = "random") +
ggpubr::stat_compare_means(label.x = 1.2, label.y.npc = "top", comparisons = list(c(1,2)), 
                           size = 5, vjust = 0.3, label = "p.format") +
    scale_fill_manual(values = c("#1874cdff",  "#ee6363ff", "#c41515ff","#eeb4b4ff")) +
scale_color_manual(values = c("dodgerblue3",  "#e54c4cff", "#aa2a2aff","#e88989ff")) +
ylab("") +
  xlab("") +
  ylim(0,NA) +
  theme_classic() +
theme(strip.background = element_blank(), panel.grid = element_blank()) + 
  theme(axis.text = element_text(color = "black"),
        axis.text.x = element_text(angle = 90),
       axis.line = element_line(color = "black", size = 0.5),
        axis.ticks.x = element_blank()) +
  ggtitle(colnames(df)[i]) + ggtheme() + NoLegend()
    
# Plot difference between Dia T0 and Dia T1 (Statistics = Dia T0 vs Dia T1)
        df_filt <- df %>% mutate(Condition = if_else(substr(patient,1,1)=="1","Dia","Ctrl")) %>% 
    mutate(Condition = paste(Condition, time_taken))  %>% 
    dplyr::filter(Condition %in% c("Dia T0", "Dia T1"))
     df_filt <- df_filt[,c(1:10,79,i)]
  colnames(df_filt) <- c(colnames(df_filt[1:10]), "Condition","value")
  df_filt$value <- as.numeric(df_filt$value)
    
df_filt <- df_filt  %>% dplyr::select(patient, time_taken, value)  %>% 
pivot_wider(names_from = time_taken, values_from = value) 
colnames(df_filt)  <- c("Patient_ID","Time0","Time1")
 df_filt <- df_filt %>% dplyr::filter(!is.na(Time1)&(!is.na(Time0)))
    df_filt <- df_filt  %>% pivot_longer(!Patient_ID, names_to = "TimePoint", values_to = "value")
    
plot2 <- df_filt  %>% 
  ggplot(aes(x = TimePoint,
             y = value)) +
geom_violin(alpha = 0.3, scale = "width") + 
stat_summary(fun = "median",
               geom = "crossbar", 
               width = 0.75,
               color = "grey30") +
geom_beeswarm(size = 3, cex = 3, 
                shape = 21, color = "black", corral = "random") +
ggpubr::stat_compare_means(label.x = 1.2, label.y.npc = "top", paired = TRUE, 
                           size = 5, vjust = 0.3, label = "p.format") +
scale_fill_manual(values = c("#ee6363ff", "#c41515ff","#eeb4b4ff")) +
scale_color_manual(values = c("#e54c4cff", "#aa2a2aff","#e88989ff")) +
ylab("") +
  xlab("") +
  ylim(0,NA) +
  theme_classic() +
theme(strip.background = element_blank(), panel.grid = element_blank()) + 
  theme(axis.text = element_text(color = "black"),
        axis.text.x = element_text(angle = 90),
       axis.line = element_line(color = "black", size = 0.5),
        axis.ticks.x = element_blank()) +
  ylim(0,NA)+
  plot_annotation(
    title = paste(stringr::str_replace_all(string = colnames(df)[i], pattern = "/", replacement = "\n")) ,
    theme = theme(plot.title = element_text(size = 8), 
                  plot.subtitle = element_text(size = 8),
                 axis.text.x = element_blank())) + ggtheme() + NoLegend()
   
  # Combine the two plots together
   plot  <- plot + plot2 
    
   # Render the plot
   print( plot)
 
   # Save the plot 
   # ggsave(paste0("../figures/cohort_characterization/sort_percentages/png/",paste0("plot_",i),".png"), 
   #    width = 12, height = 10, units = "cm", create.dir = T)
   # ggsave(paste0("../figures/cohort_characterization/sort_percentages/svg/",paste0("plot_",i),".svg"), 
   #    width = 12, height = 10, units = "cm", create.dir = T)
}

### Analysis in Age groups

The metadata contain age as well as age groups which we will use to summarize differences between age groups. 

In [ ]:
rstat_cd4  <- df  %>% 
mutate(Condition = if_else(substr(patient,1,1)=="1","Dia","Ctrl")) %>% 
dplyr::filter(time_taken == "T0")  %>% 
group_by(age_group)  %>% 
rstatix::wilcox_test(eff_to_naive_ratio_cd4~Condition, detailed = T)

In [ ]:
rstat_cd8  <- df  %>% 
mutate(Condition = if_else(substr(patient,1,1)=="1","Dia","Ctrl")) %>% 
dplyr::filter(time_taken == "T0")  %>% 
group_by(age_group)  %>% 
rstatix::wilcox_test(eff_to_naive_ratio_cd8~Condition, detailed = T)

In [ ]:
rstat_age_groups  <- rbind(rstat_cd4, rstat_cd8)

In [ ]:
rstat_age_groups

### Analysis in sex groups

The metadata contain age as well as age groups which we will use to summarize differences between age groups. 

In [ ]:
rstat_cd4_sex  <- df  %>% 
mutate(Condition = if_else(substr(patient,1,1)=="1","Dia","Ctrl")) %>% 
dplyr::filter(time_taken == "T0")  %>% 
group_by(sex)  %>% 
rstatix::wilcox_test(eff_to_naive_ratio_cd4~Condition, detailed = T)

In [ ]:
rstat_cd8_sex  <- df  %>% 
mutate(Condition = if_else(substr(patient,1,1)=="1","Dia","Ctrl")) %>% 
dplyr::filter(time_taken == "T0")  %>% 
group_by(sex)  %>% 
rstatix::wilcox_test(eff_to_naive_ratio_cd8~Condition, detailed = T)

In [ ]:
rstat_sex  <- rbind(rstat_cd4_sex, rstat_cd8_sex)

In [ ]:
rstat_sex

## Flow cytometry extracellular panel

Read the results from flow cytometry experiment. 

In [ ]:
dir.create("../figures/flow/extracellular_240604_v06/")

In [ ]:
extra_gating  <- read_xlsx("../tables/flow/240603_Extracellular.xlsx")

In [ ]:
extra_gating$Condition  <- paste(extra_gating$Disease, extra_gating$TimePoint)

Create new columns containing ratios of Naive and NonNaive

In [ ]:
extra_gating$Naive_to_Eff_CD8  <- extra_gating$CD8_Naive/extra_gating$CD8_NonNaive
extra_gating$Naive_to_Eff_CD4  <- extra_gating$CD4_Naive/extra_gating$CD4_NonNaive
extra_gating$Naive_to_Eff_CD8_10  <- extra_gating$CD8_Naive10/extra_gating$CD8_NonNaive10
extra_gating$Naive_to_Eff_CD4_10  <- extra_gating$CD4_Naive10/extra_gating$CD4_NonNaive10


In [ ]:
colnames(extra_gating)

Print plots and save. 

In [ ]:
for(i in c(6:33)){
  print(i)
  df_filt <- extra_gating %>% 
    dplyr::select(1:5,36,i)
  colnames(df_filt) <- c(colnames(df_filt[c(1:6)]), "value")
  df_filt$value <- as.numeric(df_filt$value)

# Plot difference between Dia T0 and Healthy (Statistics = Dia T0 vs HD)
   
    plot <- df_filt %>% 
  ggplot(aes(x = Condition,
             y = value)) +
geom_violin(alpha = 0.3, aes(fill = Condition), scale = "width") + 
stat_summary(fun = "median",
               geom = "crossbar", 
               width = 0.75,
               color = "grey30") +
geom_beeswarm(size = 3, aes(fill = Condition), cex = 3, 
                shape = 21, color = "black", corral = "random") +
ggpubr::stat_compare_means(label.x = 1.2, label.y.npc = "top", comparisons = list(c(1,2)), 
                           size = 5, vjust = 0.3, label = "p.format") +
    scale_fill_manual(values = c("#1874cdff",  "#ee6363ff", "#c41515ff","#eeb4b4ff")) +
scale_color_manual(values = c("dodgerblue3",  "#e54c4cff", "#aa2a2aff","#e88989ff")) +
ylab("") +
  xlab("") +
  ylim(0,NA) +
  theme_classic() +
theme(strip.background = element_blank(), panel.grid = element_blank()) + 
  theme(axis.text = element_text(color = "black"),
        axis.text.x = element_text(angle = 90),
       axis.line = element_line(color = "black", size = 0.5),
        axis.ticks.x = element_blank()) +
  ylim(0,NA)+
  plot_annotation(
    title = paste(stringr::str_replace_all(string = colnames(extra_gating)[i], pattern = "/", replacement = "\n")) ,
    theme = theme(plot.title = element_text(size = 8), 
                  plot.subtitle = element_text(size = 8),
                 axis.text.x = element_blank())) + ggtheme() + NoLegend()
    
      df_filt <- extra_gating %>% 
    filter(Condition != "0 0")   %>% 
    dplyr::select(4,5,i) 
  colnames(df_filt) <- c(colnames(df_filt[c(1:2)]), "value")
  df_filt$value <- as.numeric(df_filt$value)
df_filt <- df_filt  %>% pivot_wider(names_from = TimePoint, values_from = value) 
colnames(df_filt)  <- c("Patient_ID","Time1","Time0")
 df_filt <- df_filt %>% dplyr::filter(!is.na(Time1)&(!is.na(Time0)))
    df_filt <- df_filt  %>% pivot_longer(!Patient_ID, names_to = "TimePoint", values_to = "value")
 
# Plot difference between Dia T0 and Dia T1 (Statistics = Dia T0 vs Dia T1)
    
plot2 <- df_filt  %>% 
  ggplot(aes(x = TimePoint,
             y = value)) +
geom_violin(alpha = 0.3, scale = "width") + 
stat_summary(fun = "median",
               geom = "crossbar", 
               width = 0.75,
               color = "grey30") +
geom_beeswarm(size = 3, cex = 3, 
                shape = 21, color = "black", corral = "random") +
ggpubr::stat_compare_means(label.x = 1.2, label.y.npc = "top", paired = TRUE, 
                           size = 5, vjust = 0.3, label = "p.format") +
scale_fill_manual(values = c("#ee6363ff", "#c41515ff","#eeb4b4ff")) +
scale_color_manual(values = c("#e54c4cff", "#aa2a2aff","#e88989ff")) +
ylab("") +
  xlab("") +
  ylim(0,NA) +
  theme_classic() +
theme(strip.background = element_blank(), panel.grid = element_blank()) + 
  theme(axis.text = element_text(color = "black"),
        axis.text.x = element_text(angle = 90),
       axis.line = element_line(color = "black", size = 0.5),
        axis.ticks.x = element_blank()) +
  ylim(0,NA)+
  plot_annotation(
    title = paste(stringr::str_replace_all(string = colnames(extra_gating)[i], pattern = "/", replacement = "\n")) ,
    theme = theme(plot.title = element_text(size = 8), 
                  plot.subtitle = element_text(size = 8),
                 axis.text.x = element_blank())) + ggtheme() + NoLegend()

    plot  <- plot + plot2
    plot
   
    ggsave(create.dir = TRUE, paste0("../figures/flow/extracellular_240604_v06/",paste0("plot_",i),".png"), 
         width = 13, height = 10, units = "cm")
   ggsave(create.dir = TRUE, paste0("../figures/flow/extracellular_240604_v06/",paste0("plot_",i),".svg"), 
        width = 13, height = 10, units = "cm")
}


## Characterization of TCRgd and Cd8low cells

For characterization of Tr3-56 cells by flow cytometry, we will need to see the abundance of selected markers in different subpopulations. 

In [ ]:
extra_gating

We will be using the columns containing Geometric mean values of the intensities of CD56, TCRgd and CD16. 

In [ ]:
geomean_values  <- c("CD56","TCRgd","CD16")

In the following script, we will extract the columns containing the geomean values and visualize them per each of the populations: Naive, NonNaive, Cd8low and NK cells. 

In [ ]:
for(i in geomean_values){
    
# Extrace selected columns from the table
  df_filt <- extra_gating %>% 
    dplyr::select(1:5,30,which(grepl(colnames(extra_gating), pattern = paste0("Geometric Mean \\(",i))))
    
# Rename column names to populations
colnames(df_filt) <- c(colnames(df_filt[c(1:6)]), "CD8Naive", "CD8NonNaive", "CD8low", "NK")
 
# Reshape data.frame
    df_filt <- df_filt %>% 
    pivot_longer(cols = c(7:10), names_to = "Population", values_to = "value")

# Prepare the plot    
    plot <- df_filt %>% 
  ggplot(aes(x = Population,
             y = value)) +
geom_violin(alpha = 0.3, aes(fill = Population), scale = "width") + 
geom_beeswarm(size = 3, aes(fill = Population), cex = 3, 
                shape = 21, color = "black", corral = "random") +
stat_summary(fun = "median",
               geom = "crossbar", 
               width = 0.75,
               color = "grey30") +
ylab("") +
  xlab("") +
  ylim(0,NA) +
  theme_classic() +
theme(strip.background = element_blank(), panel.grid = element_blank()) + 
  theme(axis.text = element_text(color = "black"),
        axis.text.x = element_text(angle = 90),
       axis.line = element_line(color = "black", size = 0.5),
        axis.ticks.x = element_blank()) +
  ylim(0,NA)+
  plot_annotation(
    title = i ,
    theme = theme(plot.title = element_text(size = 8), 
                  plot.subtitle = element_text(size = 8),
                 axis.text.x = element_blank())) + ggtheme() + NoLegend()
    print(plot)
    
# Save the plot 
    ggsave(create.dir = TRUE, paste0("../figures/flow/cd8_low_Tgd_char/",paste0("plot_",i),".png"), 
         width = 13, height = 20, units = "cm")
    ggsave(create.dir = TRUE, paste0("../figures/flow/cd8_low_Tgd_char/",paste0("plot_",i),".svg"), 
         width = 13, height = 20, units = "cm")
}

## Analysis of Treg cells

First, let's load file with percentages of Treg cells from flow data. 

In [ ]:
intra02_All2  <- read_xlsx("../tables/flow/240915_Intracellular_Treg.xlsx")

In [ ]:
intra02_All2

We'll create new column Condition based on Disease and Timepoint. 

In [ ]:
intra02_All2$Condition  <- paste(intra02_All2$Disease, intra02_All2$TimePoint)

Plot the violin plots for figure. Statistics is provided to compare T1D T0 and Healthy. 

In [ ]:
for(i in c(6:9)){
  
  df_filt <- intra02_All2 %>% 
    dplyr::select(1:5,10,i)
  colnames(df_filt) <- c(colnames(df_filt[c(1:6)]), "value")
  df_filt$value <- as.numeric(df_filt$value)

plot <- df_filt %>% 
  ggplot(aes(x = Condition,
             y = value)) +
    geom_violin(alpha = 0.3, aes(fill = Condition), scale = "width") + 
stat_summary(fun = "median",
               geom = "crossbar", 
               width = 0.75,
               color = "grey30") +
geom_beeswarm(size = 3, aes(fill = Condition), cex = 3, 
                shape = 21, color = "black", corral = "random") +
ggpubr::stat_compare_means(size = 7, label = "p.format", comparisons = list(c(1,2)))+
scale_fill_manual(values = c("#1874cdff",  "#ee6363ff", "#c41515ff","#eeb4b4ff")) +
scale_color_manual(values = c("dodgerblue3",  "#e54c4cff", "#aa2a2aff","#e88989ff")) +
ylab("") +
  xlab("") +
  ylim(0,NA) +
  theme_classic() +
theme(strip.background = element_blank(), panel.grid = element_blank()) + 
  theme(axis.text = element_text(color = "black"),
        axis.text.x = element_text(angle = 90),
       axis.line = element_line(color = "black", size = 0.5),
        axis.ticks.x = element_blank()) +
  ylim(0,NA)+
  plot_annotation(
    title = paste(stringr::str_replace_all(string = colnames(intra02_All2)[i], pattern = "/", replacement = "\n")) ,
    theme = theme(plot.title = element_text(size = 8), 
                  plot.subtitle = element_text(size = 8),
                 axis.text.x = element_blank())) + ggtheme() + NoLegend()
    
  ggsave(paste0("../figures/flow/250401_treg_dia_vs_hd/",paste0("plot_",i),".png"), 
         create.dir = TRUE, width = 7.5, height = 15, units = "cm")
  ggsave(paste0("../figures/flow/250401_treg_dia_vs_hd/",paste0("plot_",i),".svg"), 
         width = 7.5, height = 10, units = "cm")
    }

Plot the violin plots for figure. Statistics is provided to compare T1D T0 and T1D T1. 

In [ ]:
for(i in c(6:29)){
  
  df_filt <- intra02_All2 %>% 
    dplyr::select(1:5,30,i)
  colnames(df_filt) <- c(colnames(df_filt[c(1:6)]), "value")
  df_filt$value <- as.numeric(df_filt$value)

plot <- df_filt %>% 
  ggplot(aes(x = Condition,
             y = value)) +
    geom_violin(alpha = 0.3, aes(fill = Condition), scale = "width") + 
stat_summary(fun = "median",
               geom = "crossbar", 
               width = 0.75,
               color = "grey30") +
geom_beeswarm(size = 3, aes(fill = Condition), cex = 3, 
                shape = 21, color = "black", corral = "random") +
ggpubr::stat_compare_means(size = 7, label = "p.format", comparisons = list(c(2,3)))+
scale_fill_manual(values = c("#1874cdff",  "#ee6363ff", "#c41515ff","#eeb4b4ff")) +
scale_color_manual(values = c("dodgerblue3",  "#e54c4cff", "#aa2a2aff","#e88989ff")) +
ylab("") +
  xlab("") +
  #ylim(0,NA) +
  theme_classic() +
theme(strip.background = element_blank(), panel.grid = element_blank()) + 
  theme(axis.text = element_text(color = "black"),
        axis.text.x = element_text(angle = 90),
       axis.line = element_line(color = "black", size = 0.5),
        axis.ticks.x = element_blank()) +
  #ylim(0,NA)+
  plot_annotation(
    title = paste(stringr::str_replace_all(string = colnames(intra02_All2)[i], pattern = "/", replacement = "\n")) ,
    theme = theme(plot.title = element_text(size = 8), 
                  plot.subtitle = element_text(size = 8),
                 axis.text.x = element_blank())) + ggtheme() + NoLegend()
    
   ggsave(paste0("../figures/flow/250401_treg_t0_vs_t1/",paste0("plot_",i),".png"), 
        create.dir = TRUE, width = 7.5, height = 16, units = "cm")
   ggsave(paste0("../figures/flow/250401_treg_t0_vs_t1/",paste0("plot_",i),".svg"), 
         width = 7.5, height = 16, units = "cm")
    }

## Flow cytometry intracellular panel

Read the data from intracellular flow cytometry.

In [ ]:
intra02_All2  <- read_xlsx("../tables/flow/240516_Intracellular_CD8.xlsx")

Create a new column with Condition. 

In [ ]:
intra02_All2$Condition  <- paste(intra02_All2$Disease, intra02_All2$TimePoint)

Plots for figure. 

In [ ]:
for(i in c(6:21)){
  df_filt <- intra02_All2 %>% 
    dplyr::select(1:5,22,i)
  colnames(df_filt) <- c(colnames(df_filt[c(1:6)]), "value")
  df_filt$value <- as.numeric(df_filt$value)

#print(df_filt)
plot <- df_filt %>% 
  ggplot(aes(x = Condition,
             y = value)) +
geom_violin(alpha = 0.3, aes(fill = Condition), scale = "width") + 
stat_summary(fun = "median",
               geom = "crossbar", 
               width = 0.75,
               color = "grey30") +
geom_beeswarm(size = 3, aes(fill = Condition), cex = 3, 
                shape = 21, color = "black", corral = "random") +
ggpubr::stat_compare_means(label.x = 1.2, label.y.npc = "top", comparisons = list(c(1,2)), 
                           size = 5, vjust = 0.3, label = "p.format") +
    scale_fill_manual(values = c("#1874cdff",  "#ee6363ff", "#c41515ff","#eeb4b4ff")) +
scale_color_manual(values = c("dodgerblue3",  "#e54c4cff", "#aa2a2aff","#e88989ff")) +
ylab("") +
  xlab("") +
  ylim(0,NA) +
  theme_classic() +
theme(strip.background = element_blank(), panel.grid = element_blank()) + 
  theme(axis.text = element_text(color = "black"),
        axis.text.x = element_text(angle = 90),
       axis.line = element_line(color = "black", size = 0.5),
        axis.ticks.x = element_blank()) +
  ylim(0,NA)+
  plot_annotation(
    title = paste(stringr::str_replace_all(string = colnames(intra02_All2)[i], pattern = "/", replacement = "\n")) ,
    theme = theme(plot.title = element_text(size = 8), 
                  plot.subtitle = element_text(size = 8),
                 axis.text.x = element_blank())) + ggtheme() + NoLegend()
    
      df_filt <- intra02_All2 %>% 
    filter(Condition != "0 0")   %>% 
    dplyr::select(4,5,i) 
  colnames(df_filt) <- c(colnames(df_filt[c(1:2)]), "value")
  df_filt$value <- as.numeric(df_filt$value)
df_filt <- df_filt  %>% pivot_wider(names_from = TimePoint, values_from = value) 
colnames(df_filt)  <- c("Patient_ID","Time1","Time0")
 df_filt <- df_filt %>% dplyr::filter(!is.na(Time1)&(!is.na(Time0)))
    df_filt <- df_filt  %>% pivot_longer(!Patient_ID, names_to = "TimePoint", values_to = "value")
    
plot2 <- df_filt  %>% 
  ggplot(aes(x = TimePoint,
             y = value)) +
geom_violin(alpha = 0.3, scale = "width") + 
stat_summary(fun = "median",
               geom = "crossbar", 
               width = 0.75,
               color = "grey30") +
geom_beeswarm(size = 3, cex = 3, 
                shape = 21, color = "black", corral = "random") +
ggpubr::stat_compare_means(label.x = 1.2, label.y.npc = "top", paired = TRUE, 
                           size = 5, vjust = 0.3, label = "p.format") +
scale_fill_manual(values = c("#ee6363ff", "#c41515ff","#eeb4b4ff")) +
scale_color_manual(values = c("#e54c4cff", "#aa2a2aff","#e88989ff")) +
ylab("") +
  xlab("") +
  ylim(0,NA) +
  theme_classic() +
theme(strip.background = element_blank(), panel.grid = element_blank()) + 
  theme(axis.text = element_text(color = "black"),
        axis.text.x = element_text(angle = 90),
       axis.line = element_line(color = "black", size = 0.5),
        axis.ticks.x = element_blank()) +
  ylim(0,NA)+
  plot_annotation(
    title = paste(stringr::str_replace_all(string = colnames(intra02_All2)[i], pattern = "/", replacement = "\n")) ,
    theme = theme(plot.title = element_text(size = 8), 
                  plot.subtitle = element_text(size = 8),
                 axis.text.x = element_blank())) + ggtheme() + NoLegend()

    plot  <- plot + plot2
    plot
  ggsave(paste0("../figures/flow/250401_intracellular/",paste0("plot_",i),".png"), 
          width = 14, height = 9.5, units = "cm")
  ggsave(paste0("../figures/flow/250401_intracellular/",paste0("plot_",i),".svg"), 
         width = 14, height = 9.5, units = "cm")
}

### Analysis in Age groups

In [ ]:
intra02_All2

In [ ]:
df1  <- intra02_All2  %>% 
dplyr::filter(TimePoint == 0)  %>% 
mutate(Disease = ifelse(Disease == 0, "Ctrl","Dia"))  %>% 
dplyr::select(patient = Patient_ID, Disease, cd4_gzmb = 17, cd8_gzmb = 19, cd8low_gzmb = 21)  %>% 
left_join(df  %>% dplyr::select(patient,age_group))  %>% 
unique()

In [ ]:
df1

In [ ]:
rstat_cd8_gzmb  <- df1  %>% 
group_by(age_group)  %>% 
rstatix::wilcox_test(cd8_gzmb~Disease, detailed = T)

In [ ]:
rstat_cd4_gzmb  <- df1  %>% 
group_by(age_group)  %>% 
rstatix::wilcox_test(cd4_gzmb~Disease, detailed = T)

In [ ]:
rstat_cd8low_gzmb  <- df1  %>% 
group_by(age_group)  %>% 
rstatix::wilcox_test(cd8low_gzmb~Disease, detailed = T)

In [ ]:
rstat_age_groups  <- rbind(rstat_age_groups, rstat_cd8_gzmb, rstat_cd4_gzmb, rstat_cd8low_gzmb)

In [ ]:
colnames(rstat_age_groups)[3]  <- "variable_tested"

### Analysis in Sex

In [ ]:
df1  <- intra02_All2  %>% 
dplyr::filter(TimePoint == 0)  %>% 
mutate(Disease = ifelse(Disease == 0, "Ctrl","Dia"))  %>% 
dplyr::select(patient = Patient_ID, Disease, cd4_gzmb = 17, cd8_gzmb = 19, cd8low_gzmb = 21)  %>% 
left_join(df  %>% dplyr::select(patient,sex))  %>% 
unique

In [ ]:
df1

In [ ]:
rstat_cd8_gzmb_sex  <- df1  %>% 
group_by(sex)  %>% 
rstatix::wilcox_test(cd8_gzmb~Disease, detailed = T)

In [ ]:
rstat_cd4_gzmb_sex  <- df1  %>% 
group_by(sex)  %>% 
rstatix::wilcox_test(cd4_gzmb~Disease, detailed = T)

In [ ]:
rstat_cd8low_gzmb_sex  <- df1  %>% 
group_by(sex)  %>% 
rstatix::wilcox_test(cd8low_gzmb~Disease, detailed = T)

In [ ]:
rstat_sex  <- rbind(rstat_sex, rstat_cd8_gzmb_sex, rstat_cd4_gzmb_sex, rstat_cd8low_gzmb_sex)

In [ ]:
colnames(rstat_sex)[3]  <- "variable_tested"

## Plot Flow differences in age groups

In [ ]:
options(repr.plot.width = 15, repr.plot.height = 4)
rstat_age_groups  %>% 
ggplot(aes(x = factor(age_group), y = estimate)) +
geom_point() +
geom_pointrange(aes(ymin = conf.low, ymax = conf.high, color = conf.low>0)) +
facet_wrap(~variable_tested, scales = "free", ncol = 5) +
scale_color_manual(values = c("grey60","darkred")) +
theme_bw() +
ggtheme()  +
geom_hline(yintercept = 0, color = 'blue')

In [ ]:
ggsave("../figures/age_groups/flow_in_age_groups.svg", create.dir = TRUE, width = 45, height = 9, units = "cm")

## Plot flow differences in sex groups

In [ ]:
options(repr.plot.width = 15, repr.plot.height = 4)
rstat_sex  %>% 
ggplot(aes(x = factor(sex), y = estimate)) +
geom_point() +
geom_pointrange(aes(ymin = conf.low, ymax = conf.high, color = conf.low>0)) +
facet_wrap(~variable_tested, scales = "free", ncol = 5) +
scale_color_manual(values = c("grey60","darkred")) +
theme_bw() +
ggtheme()  +
geom_hline(yintercept = 0, color = 'blue')

In [ ]:
ggsave("../figures/sex/flow_in_sex.svg", create.dir = TRUE, width = 45, height = 9, units = "cm")

# Correlation with scRNAseq frequencies

In this part, we will evaluate the correlation between frequencies of populations in scRNAseq dataset and the same populations measured by flow cytometry. 

Let's start by loading frequencies of CD4+ populations. 

In [ ]:
population_freq  <- read_csv("../tables/populations_freq/freq_for_model_cd4.csv")

In [ ]:
population_freq$prelim_final  <- ifelse(population_freq$Experiment_ID %in% c("Exp10", "Exp11"), "Prelim", "Final")

First, let's focus on Naive CD4+ T cells. 

### Naive CD4

In [ ]:
freq_cd4_naive  <- population_freq  %>% dplyr::filter(Level == "l2" & grepl(annotations_manual, pattern = "Naive"))  %>% 
dplyr::select(annotations_manual, Level, freq, Patient_ID, Time, prelim_final)  %>% unique  %>% 
mutate(Patient_Time = paste(Patient_ID, Time))

In [ ]:
freq_cd4_naive

The FACS percentages will be taken from the flow cytometry tables from above. 

In [ ]:
extra_gating$Patient_Time  <- paste0(extra_gating$Patient_ID, " T", extra_gating$TimePoint)

Join the FACS percentages and the scRNAseq percentages. 

In [ ]:
df2 <- extra_gating  %>% dplyr::select(Patient_Time, CD4_Naive)  %>% left_join(freq_cd4_naive)

In [ ]:
df2

Plot the correlation. 

In [ ]:
options(repr.plot.width = 8, repr.plot.height = 6)

df2 %>% 
dplyr::filter(prelim_final == "Prelim")  %>% 
ggplot(aes(x=CD4_Naive, y=freq*100)) +
  geom_point(shape = 16, size = 2, aes(color = prelim_final)) +
  geom_smooth(method=lm, alpha = 0.2) + ggtitle("scRNAseq CD4 naive vs FACS CD4 naive") + 
stat_cor(size = 7) + theme_classic() + ggtheme() +
#facet_wrap(~prelim_final, scales = "free") + 
NoLegend()

In [ ]:
ggsave("../figures/correlation_facs_scRNAseq/cd4_naive.png", 
       create.dir = TRUE,
       width = 10, height = 10, units = "cm")
ggsave("../figures/correlation_facs_scRNAseq/cd4_naive.svg", 
       width = 10, height = 10, units = "cm")

### Treg

We will repeat the process for Treg cells. 

In [ ]:
freq_cd4_treg  <- population_freq  %>% dplyr::filter(Level == "l2" & grepl(annotations_manual, pattern = "Treg"))  %>% 
dplyr::select(annotations_manual, Level, freq, Patient_ID, Time, prelim_final)  %>% unique  %>% 
mutate(Patient_Time = paste(Patient_ID, Time))

In [ ]:
freq_cd4_treg

In [ ]:
intra02_All2

In [ ]:
df2  <-  intra02_All2  %>% 
mutate(Patient_Time = paste0(extra_gating$Patient_ID, " T", extra_gating$TimePoint))  %>% 
dplyr::select(Patient_Time, CD4_Treg = 6)  %>% left_join(freq_cd4_treg)

In [ ]:
df2

In [ ]:
options(repr.plot.width = 8, repr.plot.height = 6)
 df2 %>% 
dplyr::filter(prelim_final == "Prelim")  %>% 
ggplot(aes(x=CD4_Treg, y=freq)) +
  geom_point(shape = 16, size = 2, aes(color = prelim_final)) +
  geom_smooth(method=lm, alpha = 0.2) + ggtitle("scRNAseq CD4 Treg vs FACS CD4 Treg") + 
stat_cor(size = 7) + theme_classic() + ggtheme()

In [ ]:
options(repr.plot.width = 8, repr.plot.height = 6)
 df2 %>% 
dplyr::filter(prelim_final == "Prelim")  %>% 
ggplot(aes(x=CD4_Treg, y=freq*100)) +
geom_point(shape = 16, size = 2, aes(color = factor(prelim_final))) +
 geom_smooth(method=lm) + ggtitle("") +stat_cor() + theme_classic() + ggtheme() +
 scale_y_continuous() + NoLegend() 

ggsave("../figures/correlation_facs_scRNAseq/treg.png", 
       create.dir = TRUE,
       width = 10, height = 10, units = "cm")
ggsave("../figures/correlation_facs_scRNAseq/treg.svg", 
       width = 10, height = 10, units = "cm")

### Naive CD8

We will repeat the process for Naive CD8 cells. 

In [ ]:
population_freq_cd8  <- read_csv("../tables/populations_freq/cd8_freq.csv")

In [ ]:
population_freq_cd8

In [ ]:
population_freq_cd8$prelim_final  <- ifelse(population_freq_cd8$Experiment_ID %in% c("Exp10", "Exp11"), "Prelim", "Final")

In [ ]:
md <- read_excel("../data/metadata_v07.xlsx")
df <- as.data.frame(md)
df

In [ ]:
cd8_naive  <- data.frame(pct_naive_of_CD8 = df$pct_naive_of_CD8, 
                         Patient_Time = paste(df$patient, df$time_taken))

In [ ]:
population_freq_cd8_naive  <- population_freq_cd8  %>% dplyr::filter(Level == "L2" & annot_l2 == "Naive")  %>% 
mutate(Patient_Time = paste(Patient_ID, Time))  %>% 
dplyr::select(Patient_Time, pct_from_parent, prelim_final)

In [ ]:
population_freq_cd8_naive

In [ ]:
cd8_naive  <- left_join(cd8_naive, population_freq_cd8_naive)

In [ ]:
cd8_naive

In [ ]:
options(repr.plot.width = 8, repr.plot.height = 6)
 cd8_naive %>% 
dplyr::filter(prelim_final == "Prelim")  %>% 
ggplot(aes(x=pct_naive_of_CD8, y=pct_from_parent)) +
geom_point(shape = 16, size = 2, aes(color = factor(prelim_final))) +
 geom_smooth(method=lm) + ggtitle("") +stat_cor() + theme_classic() + ggtheme() +
 scale_y_continuous() + NoLegend() 

In [ ]:
ggsave("../figures/correlation_facs_scRNAseq/cd8_naive.png", 
       create.dir = TRUE,
       width = 10, height = 10, units = "cm")
ggsave("../figures/correlation_facs_scRNAseq/cd8_naive.svg", 
       width = 10, height = 10, units = "cm")

### Tgd

We will repeat the process for Tgd cells. 

In [ ]:
cd8_tgd  <- data.frame(pct_tgd_of_CD8 = extra_gating$`FlowAIGoodEvents/Lymphocytes/Single Cells/Single Cells/Via, SSC-A subset/CD3, SSC-A subset/gdT | Freq. of Parent`, 
                         Patient_Time = paste0(extra_gating$Patient_ID, " T",extra_gating$TimePoint))

In [ ]:
population_freq_cd8

In [ ]:
population_freq_cd8_tgd  <- population_freq_cd8  %>% dplyr::filter(Level == "L1" & annot_l1 == "CD8 Unconventional T cells")  %>% 
mutate(Patient_Time = paste(Patient_ID, Time))  %>% 
dplyr::select(Patient_Time, pct_from_total, prelim_final)

In [ ]:
population_freq_cd8_tgd

In [ ]:
cd8_tgd  <- left_join(cd8_tgd, population_freq_cd8_tgd)

In [ ]:
cd8_tgd

In [ ]:
options(repr.plot.width = 8, repr.plot.height = 6)
 cd8_tgd %>% 
dplyr::filter(prelim_final == "Prelim")  %>% 
ggplot(aes(x=pct_tgd_of_CD8, y=pct_from_total)) +
geom_point(shape = 16, size = 2, aes(color = factor(prelim_final))) +
 geom_smooth(method=lm) + ggtitle("") +stat_cor() + theme_classic() + ggtheme() +
 scale_y_continuous() + NoLegend() 

In [ ]:
ggsave("../figures/correlation_facs_scRNAseq/cd8_tgd.png", 
       create.dir = TRUE,
       width = 10, height = 10, units = "cm")
ggsave("../figures/correlation_facs_scRNAseq/cd8_tgd.svg", 
       width = 10, height = 10, units = "cm")